In [1]:
!pip install git+https://github.com/openai/whisper.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-ynhrn5a2
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-ynhrn5a2
  Resolved https://github.com/openai/whisper.git to commit 6dea21fd7f7253bfe450f1e2512a0fe47ee2d258
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 KB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796926 sha256=8187946cd59eaf01cfa8b7190e803aac9eff923f455b22539a9132c8b77cbfcd
  Stored in di

In [14]:
#imports
import os
import re
import string
import numpy as np
import whisper

In [6]:
# model
# replace with model here later
model = whisper.load_model("base")

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 106MiB/s]


In [9]:
# transcribe audios using model

dir = 'audio'

transcripts = {}

for filename in os.listdir(dir):
  name = os.path.join(dir, filename)

  if os.path.isfile(name):
     if filename.endswith('.wav'):
       result = model.transcribe(name)
       id = filename.split('.')[0]
       transcripts[id] = result['text']

/usr/local/lib/python3.9/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.9/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.9/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.9/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.9/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/pytho

In [10]:
transcripts

{'84_121123_000009_000007': ' See what they have done? Cride Morrell with one hand leaning on the back of the chair and the other extending towards Valentine.',
 '84_121123_000009_000008': ' See my father, see?',
 '84_121123_000008_000004': ' Go! Do you hear?" said Villafort, while Davenie advanced to lead Moral out.',
 '84_121123_000010_000000': ' Villafort drew back and looked with astonishment on the young man, who almost a stranger to him called Nwatiya his father.',
 '84_121123_000009_000000': ' But in less than five minutes, the staircase groaned beneath an extraordinary weight.',
 '84_121123_000008_000002': ' His glance at first wandering fixed itself upon Moral.',
 '84_121123_000008_000001': ' The terrible office he had held for 25 years had succeeded in making him more or less than man.',
 '84_121123_000008_000000': ' Villafort rose half ashamed of being surprised in such a proxism of grief.',
 '84_121123_000007_000001': ' Maximilian.',
 '84_121123_000008_000003': ' Who are yo

In [12]:
# organize ground truths
ground_truths = {}

for filename in os.listdir(dir):
  name = os.path.join(dir, filename)

  if os.path.isfile(name):
    if filename.endswith('.txt') and 'original' in filename:
      id = filename.split('.')[0]
      
      with open(name) as txt:
        ground_truths[id] = txt.read()

In [13]:
ground_truths

{'84_121123_000008_000003': '"Who are you, sir," he asked, "that forget that this is not the manner to enter a house stricken with death? Go, sir, go!" But Morrel remained motionless; he could not detach his eyes from that disordered bed, and the pale corpse of the young girl who was lying on it.',
 '84_121123_000009_000007': '"See what they have done!" cried Morrel, with one hand leaning on the back of the chair, and the other extended towards Valentine.',
 '84_121123_000008_000001': 'The terrible office he had held for twenty-five years had succeeded in making him more or less than man.',
 '84_121123_000008_000004': '"Go!--do you hear?" said Villefort, while d\'Avrigny advanced to lead Morrel out.',
 '84_121123_000009_000000': 'But in less than five minutes the staircase groaned beneath an extraordinary weight.',
 '84_121123_000008_000000': 'Villefort rose, half ashamed of being surprised in such a paroxysm of grief.',
 '84_121123_000010_000000': 'Villefort drew back and looked with 

In [18]:
# Calculate number of correct transcripts
success = 0
for key, text in transcripts.items():
  truth = ground_truths[key]
  truth_seq = re.sub('['+string.punctuation+']', '', truth).split()
  seq = re.sub('['+string.punctuation+']','',text).split()
  print(truth_seq)
  print(seq)
  if truth_seq == seq:
    success += 1

['See', 'what', 'they', 'have', 'done', 'cried', 'Morrel', 'with', 'one', 'hand', 'leaning', 'on', 'the', 'back', 'of', 'the', 'chair', 'and', 'the', 'other', 'extended', 'towards', 'Valentine']
['See', 'what', 'they', 'have', 'done', 'Cride', 'Morrell', 'with', 'one', 'hand', 'leaning', 'on', 'the', 'back', 'of', 'the', 'chair', 'and', 'the', 'other', 'extending', 'towards', 'Valentine']
['See', 'my', 'father', 'see']
['See', 'my', 'father', 'see']
['Godo', 'you', 'hear', 'said', 'Villefort', 'while', 'dAvrigny', 'advanced', 'to', 'lead', 'Morrel', 'out']
['Go', 'Do', 'you', 'hear', 'said', 'Villafort', 'while', 'Davenie', 'advanced', 'to', 'lead', 'Moral', 'out']
['Villefort', 'drew', 'back', 'and', 'looked', 'with', 'astonishment', 'on', 'the', 'young', 'man', 'who', 'almost', 'a', 'stranger', 'to', 'him', 'called', 'Noirtier', 'his', 'father']
['Villafort', 'drew', 'back', 'and', 'looked', 'with', 'astonishment', 'on', 'the', 'young', 'man', 'who', 'almost', 'a', 'stranger', 'to', 

In [17]:
# Calculate accuracy
accuracy = success / len(ground_truths.keys())
print(accuracy)

0.3


### Notes
- Evaluation is hard. What about slight differences?
- Ground truths are possibly noisy.